# ENGR 225 - COMPUTATIONAL METHODS I


## Welcome

Welcome to ENGR 225 - Computational Methods I. It is the first course of the Computational Methods course series (three courses in total). It is designed for students with no or little programming experience. You will be learning the basics of programing in Python and linear algebra in this course.

My name is Bonnie Ludka, and I am your instructor for this course. Please click this link to view my department profile: https://engineering.humboldt.edu/people/bonnie-ludka

If you have any questions, please feel free to contact me by sending me a message using the Canvas Inbox.

## Fun with Programing in Python and Application of Linear Algebra

We will learn the basics of **programing in Python** and **linear algebra** in this course. Something cool like the example below: the Julia Set.

**The math:**

Julia set fractals are normally generated by initializing a complex number  $z = x + yi$ where $i^2 = -1$ and $x$ and $y$ are image pixel coordinates in the range of about -2 to 2. Then, $z$ is repeatedly updated using: $z = z^2 + c$ where $c$ is another complex number that gives a specific Julia set. After numerous iterations, if the magnitude of $z$ is less than 2 we say that pixel is in the Julia set and color it accordingly. Performing this calculation for a whole grid of pixels gives a fractal image.

Details: https://www.karlsims.com/julia.html

**The output of the Python Code**

In [ ]:
#@title
# Julia Set - varying iterations, constant C
#@title
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython import display
from IPython.display import HTML

x_lim = 2 # limit of x
y_lim = 2 # limit of y
n_grid = 512

n_it_max = 80
z_max = 2

im_arr_list = []

n_output = 2
for k in range(n_it_max):
    if k%n_output==0:
        im_arr = np.zeros([n_grid, n_grid])
        for i in range(n_grid):
            for j in range(n_grid):
                x = i/n_grid*(2*x_lim) + (-x_lim)
                y = j/n_grid*(2*y_lim) + (-y_lim)

                z = complex(x, y)

                n_it = 0
                while abs(z) <= z_max and n_it < k:
                    c = complex(-0.79, 0.15) # c can vary as well.
                    z = z**2 + c
                    n_it += 1
                ratio = n_it / n_it_max
                im_arr[i,j] = ratio

        im_arr_list.append(im_arr)

# make the animation
fig, ax = plt.subplots(1, figsize=[8,8])

def drawframe(n):
    if n%1==0:
        ax.clear()
        ax.set_title("Iteration={}.".format(n*n_output))
        plt.axis('off')
        ax.imshow(im_arr_list[n], cmap="hot")

anim = animation.FuncAnimation(fig, drawframe, frames=n_it_max//n_output, interval=50, blit=False)
html = HTML(anim.to_html5_video())
display.display(html)
plt.close()

**Or**

In [ ]:
#@title
# #@title
# # Julia Set - constant iterations, varying C

x_lim = 2 # limit of x
y_lim = 2 # limit of y
n_grid = 512
im_arr = np.zeros([n_grid, n_grid])
n_it_max = 50
z_max = 2
n_time = 100


im_arr_list = []

n_output = 1
for k in range(n_time):
    if k%n_output==0:
        im_arr = np.zeros([n_grid, n_grid])
        for i in range(n_grid):
            for j in range(n_grid):
                x = i/n_grid*(2*x_lim) + (-x_lim)
                y = j/n_grid*(2*y_lim) + (-y_lim)

                z = complex(x, y)

                n_it = 0
                while abs(z) <= z_max and n_it < n_it_max:
                    c = complex(-0.79, np.cos(k*2*np.pi/20)*0.2) # c can vary as well.
                    z = z**2 + c
                    n_it += 1
                ratio = n_it / n_it_max
                im_arr[i,j] = ratio

        im_arr_list.append(im_arr)

# make the animation
fig, ax = plt.subplots(1, figsize=[8,8])

def drawframe(n):
    if n%1==0:
        ax.clear()
        plt.axis('off')
        ax.imshow(im_arr_list[n], cmap="hot")

anim = animation.FuncAnimation(fig, drawframe, frames=n_time//n_output, interval=200, blit=False)
html = HTML(anim.to_html5_video())
display.display(html)
plt.close()

## Another Engineering Example: Heat Diffusion/Condution in 2D

The equation to describe the heat diffusion is given below:

>$
\frac{\partial T}{\partial t} = \alpha \nabla^2 T
$

where $\alpha$ is the thermal diffusivity.

In a 2D space, the equation can be written:

>$
\frac{\partial T}{\partial t} = \alpha \frac{\partial^2 T}{\partial x} + \alpha \frac{\partial^2 T}{\partial y}
$

The explicit form of the discrtized equation is:

>$
T_{i,j}^{n+1} = T_{i,j}^n + \frac{\alpha \Delta t}{\Delta x^2}(T_{i+1,j}^n - 2 T_{i,j}^n + T_{i-1,j}^n) + \frac{\alpha \Delta t}{\Delta y^2}(T_{i,j+1}^n-2 T_{i,j}^n + T_{i,j-1}^n)
$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from skimage import io
import cv2

# read in the image for assgining the initial temperature

img_gray_shrink = np.zeros((200,200))

# font
text = "CSU"
font = cv2.FONT_HERSHEY_SIMPLEX
org = (10, 150)
fontScale = 1.5
color = (255, 0, 0)
thickness = 10
cmap = cm.Greens

# Using cv2.putText() method
image = cv2.putText(img_gray_shrink, text, org, font,
                   fontScale, color, thickness, cv2.LINE_AA)

# meshing
row, col = img_gray_shrink.shape
n_x = col
n_y = row

W = 0.1 # width of the domain
H = W*row/col # hight of the domain

x = np.linspace(0, W, n_x)
dx = x[1]-x[0]
y = np.linspace(0, H, n_y)
X, Y = np.meshgrid(x, y)
dy = y[1]-y[0]

t_total = 10
dt = 0.01
n_ts = int(t_total/dt)+1

# initial T
T = np.zeros([n_y, n_x, n_ts]) # to store temperature field for every timestep
T[:,:,0] = img_gray_shrink/255.0

# property
alpha = 1.0e-6

# the solver
for i_ts in range(1, n_ts):
    T[1:-1, 1:-1, i_ts] = (T[1:-1,1:-1,i_ts-1] + alpha * dt / dx**2 * (T[1:-1, 2:,i_ts-1] - 2 * T[1:-1, 1:-1,i_ts-1] + T[1:-1, 0:-2,i_ts-1]) \
                           + alpha * dt / dy**2 * (T[2:,1: -1,i_ts-1] - 2 * T[1:-1, 1:-1,i_ts-1] + T[0:-2, 1:-1,i_ts-1])
                          )
    T[0,:,i_ts] = 0
    T[-1,:,i_ts] = 0
    T[:,0,i_ts] = 0
    T[:,-1,i_ts] = 0

# visualization
every_n = 10

T_ani = T[:,:, ::every_n]

from matplotlib import animation
from IPython import display
from IPython.display import HTML

fig, ax = plt.subplots(1, figsize=[8,8])
txt_title = ax.set_title('')
ax.set_xticks([])
ax.set_yticks([])
plt.axis('off')

def drawframe(n):
    ax.clear()
    surf = ax.contourf(X, Y, T_ani[:,:,n], levels=np.linspace(0.1,1.1,10), cmap=cmap)
    ax.set_title("time = "+str(round(n*dt*every_n, 2))+" s")
    plt.gca().invert_yaxis()
    plt.axis('off')
    return surf

# blit=True re-draws only the parts that have changed.
anim = animation.FuncAnimation(fig, drawframe, frames=int(n_ts/every_n), interval=30, blit=False)

from IPython.display import HTML
html = HTML(anim.to_html5_video())

display.display(html)
plt.close()

---

End of document

$f(x) = y\times h$